In [ ]:
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time 
import win32api #funciona apenas em sistemas windows
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

In [10]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
#options.add_argument('--headless') # executa sem abrir a janela
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--start-maximized')
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

driver.implicitly_wait(30)
#driver.set_window_size(1440, 900)

#from google.colab import data_table #permite filtrar a tabela na própria linha de resposta
#data_table.enable_dataframe_formatter()

In [11]:
def Aplicar():
    aplicar = driver.find_element(By.CSS_SELECTOR, 'button[id="aplicar"][type="button"]') #Aplica os filtros' 
    driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'nearest'})", aplicar)
    time.sleep(1)
    aplicar.click()

def Expande_Lista(nome_lista:str):
    time.sleep(2) #Aguarda a página carregar os elementos
    lista = driver.find_element(By.CSS_SELECTOR, 'ng-select[id="' + nome_lista + '"]''') #Encontra o elemento 'lista' no DOM
    driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'nearest'})", lista) #coloca o elemento no centro da tela para que o comando click() funcione
    time.sleep(1)
    lista.click() #O comando click() só funciona se o elemento estiver visível na tela
    time.sleep(2)

def Executa_filtro(nome_lista:list):
    for itens in nome_lista:
        busca_itens = driver.find_element(By.CSS_SELECTOR,'div[title=' + '"' + f"{itens}" + '"' + '][class="ng-star-inserted"]')
        busca_itens.click()
        time.sleep(2)
    Aplicar()
    
def Coleta_links():
    driver.implicitly_wait(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    has_pagination = len(driver.find_elements(By.XPATH,'//*[@id="main-content"]/pncp-list/pncp-results-panel/pncp-tab-set/div/pncp-tab[1]/div/div[2]/div[2]/div[3]/pncp-pagination[*]')) > 0
    aux = 1
    urls=[]

    if has_pagination:
       max_pagination = max({int(element.string) for element in soup.find_all('button','page')})
    else:
        max_pagination = 0
    
    while aux <= int(max_pagination) or aux == 1:
        elements = driver.find_elements(By.XPATH, '//*[@id="main-content"]/pncp-list/pncp-results-panel/pncp-tab-set/div/pncp-tab[1]/div/div[2]/div[2]/div[2]/pncp-items-list/div')
        for element in elements:
            inicio = element.text[21:35]
            meio = element.text[-4:]
            fim = re.sub("^0+", "",element.text[39:44])
            urls.append('https://pncp.gov.br/app/editais/' + inicio + '/' + meio + '/' + fim)
        aux += 1
        if has_pagination:
            pag = driver.find_elements(By.CSS_SELECTOR, 'button[class="page"][type="button"]')
            for item in pag: #busca o próximo link
                if item.text == str(aux):
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'nearest'})", item)
                    time.sleep(1)
                    item.click()
                    break
    return urls

def Coleta_Dados(cabecalho):
    aux = True #garante que o while seja executado ao menos uma vez
    linha = {}
    descricao = []
    qtd = []
    valor_unitario = []
    valor_total = []
    btn_prox_pag = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Ir para próxima página"]')
    btn_pag_ant = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Ir para página anterior"]')
    while aux == True:
        dados = [item.text for item in driver.find_elements(By.XPATH, '//*[@id="main-content"]/pncp-item-detail/div/pncp-tab-set/div/pncp-tab[1]/div/div/pncp-table/div/ngx-datatable/div/datatable-body/datatable-selection/datatable-scroller/datatable-row-wrapper[*]/datatable-body-row/div[2]/datatable-body-cell[position() > 1]/div/span')]
        descricao.extend(dados[0::4])
        qtd.extend(dados[1::4])
        valor_unitario.extend(dados[2::4])
        valor_total.extend(dados[3::4])
        if btn_prox_pag.is_enabled():
            driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'nearest'})", btn_prox_pag)
            time.sleep(1)
            btn_prox_pag.click()
            time.sleep(1)
            continue
        if not btn_prox_pag.is_enabled() and btn_pag_ant.is_enabled():
            aux = False
        if not btn_prox_pag.is_enabled() and not btn_pag_ant.is_enabled():
            aux = False
    
    local = driver.find_element(By.XPATH, '//*[@id="main-content"]/pncp-item-detail/div/div[3]/div[1]/p/span').text #Coleta a cidade/estado
    url = driver.current_url #Coleta a url 

    linha = dict(**dict(zip(['Descrição'], [descricao])), **dict(zip(['Qtd'], [qtd])), **dict(zip(['Valor unit estimado'], [valor_unitario])), **dict(zip(['Valor Total estimado'], [valor_total])))
    linha = pd.DataFrame(linha)
    df = pd.concat([cabecalho, linha])
    df['URL'].fillna(value=url, inplace=True)
    df['Local'].fillna(value=local, inplace=True)
    return df

In [14]:
#Opções para status: recebendo_proposta; propostas_encerradas; todos; encerradas
#Opções para Estado: PR, RS, SC, etc...
#Opções para modalidades: Dispensa de Licitação, Pregão eletrônico 

status = 'recebendo_proposta' 
ufs = ['RS']
municipios = ['Alto Feliz']
modalidades = ['']
q = ''

url = f"https://pncp.gov.br/app/editais?q="

if status == 'todos' or status == 'encerradas' and (q == '' or len(q) < 3):
    win32api.MessageBox(0, "Para selecionar 'todos' você precisa digitar uma palavra chave com mais de 3 caracteres", "AVISO!")
    driver.quit()
if status != '':
    url = url + f"{q}&status={status}&pagina=1"
    driver.get(url)
    
    if '' not in municipios:  
        Expande_Lista('municipios')

        for municipio in municipios:
            Executa_filtro(municipios)
          
    if '' in municipios and '' not in ufs:
        Expande_Lista('ufs')

        for estado in ufs:
            Executa_filtro(ufs)

    if '' not in modalidades:
        Expande_Lista('modalidades')

        for modalidade in modalidades:
            Executa_filtro(modalidades)

soup = BeautifulSoup(driver.page_source, 'html.parser')

if len(soup.find_all('pncp-item-not-found')) > 0: 
     print("Não existem itens!")
     driver.quit()

links = Coleta_links()

cabecalho = pd.DataFrame(columns=['URL','Local','Descrição', 'Qtd', 'Valor unit estimado', 'Valor Total estimado'])
df = pd.DataFrame()
for item in links:
    driver.get(item)
    time.sleep(1)

    df = pd.concat([df, Coleta_Dados(cabecalho)])
df

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"button[aria-label="Ir para próxima página"]"}
  (Session info: chrome=121.0.6167.140)
Stacktrace:
	GetHandleVerifier [0x01101673+52979]
	(No symbol) [0x01087961]
	(No symbol) [0x00F6DD3D]
	(No symbol) [0x00FA5FBB]
	(No symbol) [0x00FA60FB]
	(No symbol) [0x00FDCF92]
	(No symbol) [0x00FC4534]
	(No symbol) [0x00FDB3FE]
	(No symbol) [0x00FC4286]
	(No symbol) [0x00F9C063]
	(No symbol) [0x00F9CECD]
	GetHandleVerifier [0x01418D83+3294723]
	GetHandleVerifier [0x01456CC2+3548482]
	GetHandleVerifier [0x01451C9C+3527964]
	GetHandleVerifier [0x0119870E+671630]
	(No symbol) [0x01091EB4]
	(No symbol) [0x0108D808]
	(No symbol) [0x0108D92D]
	(No symbol) [0x0107F7E0]
	BaseThreadInitThunk [0x763D7BA9+25]
	RtlInitializeExceptionChain [0x7767BD2B+107]
	RtlClearBits [0x7767BCAF+191]


In [ ]:
cabecalho

In [9]:
driver.quit()